# Libraries

In [10]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Import data

In [12]:
%run Data_preprocessing.ipynb

************************************************************
There are 6 set of X
X_*_nozero
------------------------------------------------------------
Normalized version
X_*_norm
------------------------------------------------------------
There are 4 set of Y
y_train_t1, y_train_t2, y_valid_t1, y_valid_t2
when training, please use: 'y_train_t1_value,y_train_t2_value,y_valid_t1_value,y_valid_t2_value'
************************************************************


In [13]:
print(X_train_norm.shape)
print(X_valid_norm.shape)
print(X_test_norm.shape)

(16760, 61)
(2394, 61)
(4790, 61)


# Hyperparameters

In [69]:
input_size = 61# 7488/24
output_size = 1 
hidden_size=256
learning_rate = 0.01
batch_size = 64
num_epochs = 2

# Create CNN

In [24]:
class CNN(nn.Module):
    def __init__(self, input_size, output_size):
        super(CNN, self).__init__()
        self.convlayer = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=6, kernel_size=1, stride=1, padding=0, dilation=0),
            nn.ReLU(),
            nn.Conv1d(in_channels=6, out_channels=1, kernel_size=1, stride=1, padding=0, dilation=0),
            nn.ReLU(),
        )
        
        self.fclayers = nn.Sequential(
            nn.Linear(input_size, 30),
            nn.Linear(30, output_size),
        )
        
        #self.dropout = nn.Dropout(p=0.5)
        
    def forward(self, x):
         x = self.convlayer(x)
         x = x.view(-1,61)
         x = self.fclayers(x)
         #x = torch.argmax(x)
         return x

# Data transform

In [25]:
class CusDatasetLoader(Dataset):
    def __init__(self,x,y):
        self.len = y.shape[0]
        self.x_data = x
        self.y_data = y
  
    def __len__(self):
        return self.len

    def __getitem__(self, index):
        #return self.dataframe.iloc[index]
        return self.x_data[index], self.y_data[index]

In [26]:
# X_shape(X) : used for split the data into a right tensor size
#input: X in shape of 16760 rows × 104 columns (df)
#output: X in shape of torch.Size([16760, 104, 1]) (tensor)
def X_tensor(X):
    X_tensor = torch.from_numpy(np.array(X)).to(torch.float32)
    X_tensor = torch.split(X_tensor,1,1)
    X_tensor = torch.stack(X_tensor).permute(1, 0, 2)
    print("X now in shape of",X_tensor.shape)
    return X_tensor

# y_tensor(y) : used for split the data into a right tensor size
#input: X in shape of 16760 rows × 7488 columns (df)
#output: X in shape of torch.Size([16760, 312, 24]) (tensor)
def y_tensor(y):
    y= torch.from_numpy(np.array(y)).to(torch.float32).reshape(len(y),1)
    print("y now in shape of",y.shape)
    return y

In [60]:
# X should be in smaples, channel, rows
X1 = X_tensor(X_train_norm)
y2 = y_tensor(y_train_t2.iloc[:,-1])

train_datasets = CusDatasetLoader(X1, y2)
train_loader = DataLoader(dataset=train_datasets, batch_size=batch_size, shuffle=True)

x,y = train_datasets[0]
print(x.shape)
print(y.shape)

X now in shape of torch.Size([16760, 61, 1])
y now in shape of torch.Size([16760, 1])
torch.Size([61, 1])
torch.Size([1])


# Model train

In [66]:
model = CNN(input_size, output_size).to(device)

In [67]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [68]:
losses=[]
acc_list=[]

for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        num_correct = 0
        num_samples = 0
        data = data.to(device=device)
        targets = targets.to(device=device)
        #print(targets)
        #targets = torch.squeeze(targets, 1).type(torch.LongTensor)
        
        # forward
        scores = model(data)
        #print(scores)
        loss = criterion(scores, targets)
        
        l2_lambda = 5e-05
        l2_norm = sum(p.pow(2.0).sum()
                  for p in model.parameters())
 
        loss = loss + l2_lambda * l2_norm
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        
        
        # gradient descent or adam step
        optimizer.step()
        
    if epoch%5 == 0:
        losses.append(loss.detach().numpy() )
    print(f'Epoch {epoch:03}: | Loss: {loss:.5f} | Acc: {acc:.3f}')
        
        

RuntimeError: Given groups=1, weight of size [6, 1, 1], expected input[64, 61, 1] to have 1 channels, but got 61 channels instead

# Model Accuracy

In [51]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0

    # Set model to eval
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).permute(0,2,1)
            y = torch.squeeze(y.to(device=device), 1)

            scores = model(x)
            _, predictions = scores.max(1)
            #print(predictions)
            #print(torch.count_nonzero(predictions))
            num_correct += int((predictions == y).sum())
            num_samples += predictions.size(0)
            
    # Toggle model back to train
    model.train()
    #print(num_correct , num_samples)
    return num_correct / num_samples

In [52]:
print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:2f} %")
#print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f} %")

Accuracy on training set: 0.000000 %


In [53]:
X2 = X_tensor(X_valid_norm)
y2 = y_tensor(y_valid_t1.iloc[:,-1])

Valid_datasets = CusDatasetLoader(X2, y2)
Valid_loader = DataLoader(dataset=train_datasets, batch_size=batch_size, shuffle=True)
print(f"Accuracy on test set: {check_accuracy(Valid_loader, model)*100:.2f} %")

X now in shape of torch.Size([2394, 61, 1])
y now in shape of torch.Size([2394, 1])
Accuracy on test set: 0.00 %


# Test

In [54]:
def test(loader, model):
    # Set model to eval
    model.eval()
    res = torch.tensor([], dtype=torch.int64)
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            print(x.shape)
            y = torch.squeeze(y.to(device=device), 1)
            print(y.shape)

            scores = model(x)
            _, predictions = scores.max(1)
            res =  torch.cat((predictions, res), 0)  
    model.train()
   
    return res

In [ ]:
X_test_mean
x_valid_mean=X_test_mean.values
x_valid_mean_scaled = scaler.fit_transform(x_valid_mean)
X_test_norm=pd.DataFrame(x_valid_mean_scaled)
X_test_norm

In [ ]:
X_test_final = X_tensor(X_test_norm).permute(0,2,1)
y_final = torch.zeros([4790,1])

test_datasets = CusDatasetLoader(X_test_final, y_final)
test_loader = DataLoader(dataset=test_datasets, batch_size=batch_size, shuffle=True)

result = test(test_loader, model)
result

In [ ]:

#df.index = ['Row_1', 'Row_2', 'Row_3', 'Row_4']
res1 = pd.DataFrame(result, columns=["Predicted"])
res1.index = X_test.index
res1.index.name = 'Id'
pd.DataFrame(res1).to_csv('out.csv')
print(torch.count_nonzero(torch.from_numpy(np.array(res1)).to(torch.float32)))
res1

#

#

#